From Geo-Python 2019
Exercise 6 https://github.com/geo-python-2019/Exercise-6/blob/master/Exercise-6-problems-1-3.ipynb

Question 1

Read the data file into a variable called data


In [ ]:
import pandas as pd
import numpy as np

filepath = r"C:\Users\ash c\Desktop\python\1091402.txt"
data = pd.read_csv(filepath, delim_whitespace=True, na_values='-9999', usecols=["STATION", "ELEVATION", "LATITUDE", "LONGITUDE","DATE","PRCP","TAVG","TMAX","TMIN"], skiprows=[1])

Calculate how many no-data values there are for the TAVG and TMIN columns

In [ ]:
#prints the number of no-data values in the TAVG column
tavg_nodata_ct = len(data['TAVG']) - len(data['TAVG'].dropna())
print(tavg_nodata_ct)

In [ ]:
#prints the number of no-data values in the TMIN column
tmin_nodata_ct = len(data['TMIN']) - len(data['TMIN'].dropna())
print(tmin_nodata_ct)

Calculate the total number of days covered by this data file
Find the date of the first observation

In [ ]:
#prints the total number of days
dayct = len(data["DATE"])
print(dayct)

In [ ]:
#prints date of first observation
first_obs = data["DATE"].at[1]
print(first_obs)

Find the average T for the whole data file from column TAVG


In [ ]:
#prints average temperature for the entire time period
avg_temp = data["TAVG"].mean()
print(avg_temp)

Find the average TMAX temperature over the summer of 1969 (May, June July, August)

In [ ]:
#creates a column each for the time as a string and datetime
data["TIMESTR"] = data["DATE"].astype(str)
data["YRMODA"] = pd.to_datetime(data["TIMESTR"])
data["YEAR"] = data["YRMODA"].dt.year
data["MONTH"] = data["YRMODA"].dt.month

grouped = data.groupby(["YEAR", "MONTH"])

#specify key of wanted months
sgroup = [(1969, 5), (1969, 6), (1969, 7), (1969, 8)] 

#create empty array
mean = [] 
#start for loop to calculate mean of group
for key, group in grouped: 
    if key in sgroup:
        #use getgroup to select each group in the list of keys
        groups = grouped.get_group(key)
        #add the calculated mean of the group to the empty mean array
        mean.append(groups["TMAX"].mean())

#prints the mean of the mean monthly temperatures
print(np.mean(mean))
print(grouped.groups.keys())

Problem 2

Calculate monthly average temperatures in degrees Celsius from the daily values in the data file. 

In [ ]:
#runs function to convert the temperatures from F to C
def tempconvert(temp):
    output_temp = (temp - 32) / 1.8
    return output_temp

In [ ]:
#calculate monthly avg temps
select = data.filter(["TAVG", "YEAR", "MONTH"])
#use double bracket to create without copy
    #interior brackets are for list and outer are for indexing operator
    #single bracket for series and double bracket for dataframe
#use filter to create a copy
newcol = {'TAVG': 'F'}
select = select.rename(columns=newcol)
select["C"] = select["F"].apply(tempconvert)
monthlydata = select.groupby(["YEAR","MONTH"])

meancols = ["F", "C"]

monthlyavg = pd.DataFrame()
for key, group in monthlydata:
    #finds the mean for each group in the columns F and C
    meanvals = group[meancols].mean() 
    meanvals["YRMO"] = key
    monthlyavg = monthlyavg.append(meanvals, ignore_index=True)
    
time = pd.DatmaFrame(monthlyavg["YRMO"].values.tolist(), columns=["YEAR","MONTH"])
monthlyavg["YEAR"] = time["YEAR"]
monthlyavg["MONTH"] = time["MONTH"]

Problem 3

Calculate monthly temperature anomalies:
- Calculate a mean temperature for each month over the period from 1952 to 1980
- Calculate a temperature anomaly for every month 

In [ ]:
datarange = select.loc[(select["YEAR"] >= 1952) & (select["YEAR"] <= 1980)]
rangegroup = datarange.groupby(["MONTH"])

reftemps = pd.DataFrame()
for key, group in rangegroup:
    meanvals = group[meancols].mean()
    reftemps = reftemps.append(meanvals, ignore_index=True)
    
reftemps["MONTH"] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
reftemps = reftemps[["MONTH", "C", "F"]]
print(reftemps)

In [ ]:
diff = monthlyavg.merge(reftemps, on="MONTH")
diff["DIFF"] = diff["C_x"] - diff["C_y"] 

newwcols = {"C_x": "TEMP_C", "F_x": "TEMP_F", "C_y": "REFTEMP_C", "F_y": "REFTEMP_F"}
diff = diff.rename(columns=newwcols)

diff.to_csv("helsinki.csv", sep=',', index=False, float_format="%.1f") 